In [5]:
from datasets import load_dataset
from transformers import pipeline

## Load the pubmed summarization dataset

In [2]:
dataset = load_dataset("ccdv/pubmed-summarization")

Generating train split: 119924 examples [00:17, 7015.80 examples/s]
Generating validation split: 6633 examples [00:00, 6944.02 examples/s]
Generating test split: 6658 examples [00:00, 7258.71 examples/s]


In [3]:
print(f"Features: {dataset['train'].column_names}")

Features: ['article', 'abstract']


In [4]:
sample = dataset['train'][1]
print(f"""
Article (excerpt of 500 characters, total length: {len(sample["article"])}):
""")
print(sample['article'][:500])
print(f'\nSummary (length: {len(sample["abstract"])}):')
print(sample["abstract"])


Article (excerpt of 500 characters, total length: 18281):

it occurs in more than 50% of patients and may reach 90% in certain types of cancers , especially in patients undergoing chemotherapy and/or radiation therapy.1 anemia is defined as an inadequate circulating level of hemoglobin ( hb ) ( hb < 12 g / dl ) and may arise as a result of the underlying disease , bleeding , poor nutrition , chemotherapy , or radiation therapy . 
 preliminary studies suggest that survival and loco - regional control after radiation therapy , especially in head and neck 

Summary (length: 2010):
backgroundanemia in patients with cancer who are undergoing active therapy is commonly encountered and may worsen quality of life in these patients . the effect of blood transfusion is often temporary and may be associated with serious adverse events . 
 erythropoiesis - stimulating agents are not effective in 30%50% of patients and may have a negative effect on overall survival.aimsto assess the efficacy and f

## Evaluate pretrained models

In [ ]:
summarizer = pipeline("summarization", model="Falconsai/medical_summarization")